This notebook replicates code posted in an NI4AI [blog post](https://blog.ni4ai.org/post/2020-03-30-sunshine-data/) written by Sascha von Meier. The code provides a basic exploration of the "Sunshine" data hosted in the NI4AI platform.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from tabulate import tabulate

import btrdb
from btrdb.utils.timez import ns_delta, ns_to_datetime, to_nanoseconds

In [2]:
#Establishing the connecting to the server
db = btrdb.connect()


In [11]:
#Viewing the collection: Data streams are organized into a hierarchichal tree of collections, so let’s view the collections available for the Sunshine dataset using the list_collections method.
db.list_collections("sunshine")


['sunshine/PMU1']

In [10]:
#Choosing streams; we are choosing PMU3 here.
streams = db.streams_in_collection('sunshine/PMU3')
streams

[]

In [5]:
#Retrieving Stream Data
def describe_streams(streams):
    table = [["Index", "Collection", "Name", "Units", "UUID"]]
    for idx, stream in enumerate(streams):
        tags = stream.tags()
        table.append([
            idx, stream.collection, stream.name, tags["unit"], stream.uuid
        ])
    return tabulate(table, headers="firstrow")

print(describe_streams(streams))

  Index  Collection     Name    Units    UUID
-------  -------------  ------  -------  ------------------------------------
      0  sunshine/PMU1  LSTATE  mask     6ffb2e7e-273c-4963-9143-b416923980b0
      1  sunshine/PMU1  C1ANG   deg      d625793b-721f-46e2-8b8c-18f882366eeb
      2  sunshine/PMU1  C3MAG   amps     fb61e4d1-3e17-48ee-bdf3-43c54b03d7c8
      3  sunshine/PMU1  C2MAG   amps     d765f128-4c00-4226-bacf-0de8ebb090b5
      4  sunshine/PMU1  C1MAG   amps     1187af71-2d54-49d4-9027-bae5d23c4bda
      5  sunshine/PMU1  C3ANG   deg      0be8a8f4-3b45-4fe3-b77c-1cbdadb92039
      6  sunshine/PMU1  L3ANG   deg      e4efd9f6-9932-49b6-9799-90815507aed0
      7  sunshine/PMU1  L2ANG   deg      886203ca-d3e8-4fca-90cc-c88dfd0283d4
      8  sunshine/PMU1  L3MAG   volts    b2936212-253e-488a-87f6-a9927042031f
      9  sunshine/PMU1  L1ANG   deg      51840b07-297a-42e5-a73a-290c0a47bddb
     10  sunshine/PMU1  C2ANG   deg      97de3802-d38d-403c-96af-d23b874b5e95
     11  sunshine/

In [6]:
#Choose a time window
start = to_nanoseconds('2015-08-15 3:42:00')
end = to_nanoseconds('2015-08-15 3:47:00')

In [9]:
#Retrieve values from stream 4 (current magnitude for PMU 1)
s = streams[4]
data = s.values(start, end)
data[:2]

[]

Values are returned as a list of tuples reporting RawPoint values and the associated version. Each RawPoint object includes a time stamp (in nanoseconds) and a measurement value.

(RawPoint(time_stamp, value), version)

In [8]:
# Retrieve only the RawPoint objects (without the version)
points, _ = zip(*data)

# Unpack raw points into time stamps and values
times, values = zip(*points)

# Convert time stamps (in nanoseconds) to datetime objects
dtimes = [ns_to_datetime(t) for t in times]

# Convert to a Series, using the time stamp as an index
series = pd.Series(values, index=times)

ValueError: not enough values to unpack (expected 2, got 0)

In [ ]:
# Graph time series measurements
series.plot()
plt.xlabel('Time Stamp')
plt.ylabel('Phase 1 Current Magnitude')
plt.show()